<a href="https://colab.research.google.com/github/githarshgit/crop_recommendation/blob/main/Crop_Recommendation_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install scikit-learn pandas numpy joblib


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
import joblib



In [12]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/CROP.csv')   # rename file if needed
df.head()


,N,P,K,temperature,humidity,ph,rainfall,label,Urea,SSP,MOP,TSP,DAP
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,195.65,262.50,71.67,0.0,0.0
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,184.78,362.50,68.33,0.0,0.0
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,130.43,343.75,73.33,0.0,0.0
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,160.87,218.75,66.67,0.0,0.0
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,169.57,262.50,70.00,0.0,0.0


In [13]:
df.info()
df.describe()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
 8   Urea         2200 non-null   float64
 9   SSP          2200 non-null   float64
 10  MOP          2200 non-null   float64
 11  TSP          2200 non-null   float64
 12  DAP          2200 non-null   float64
dtypes: float64(9), int64(3), object(1)
memory usage: 223.6+ KB


,0
N,0
P,0
K,0
temperature,0
humidity,0
ph,0
rainfall,0
label,0
Urea,0
SSP,0


In [14]:
df.fillna(df.median(numeric_only=True), inplace=True)


In [15]:
le = LabelEncoder()
df['label_enc'] = le.fit_transform(df['label'])


In [16]:
X_crop = df[['N','P','K','temperature','humidity','ph','rainfall']]
y_crop = df['label_enc']


In [17]:
X_fert = X_crop.copy()   # same features
y_fert = df[['Urea','SSP','MOP','TSP','DAP']]


In [18]:
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_crop, y_crop, test_size=0.2, random_state=42, stratify=y_crop)

Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    X_fert, y_fert, test_size=0.2, random_state=42)


In [19]:
crop_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    class_weight='balanced',
    random_state=42
)

crop_model.fit(Xc_train, yc_train)


RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=300,
                       random_state=42)

In [20]:
pred_crop = crop_model.predict(Xc_test)
acc = accuracy_score(yc_test, pred_crop)

print("Crop Model Accuracy:", acc)
print("\nClassification Report:\n")
print(classification_report(yc_test, pred_crop, target_names=le.classes_))


Crop Model Accuracy: 0.9931818181818182

Classification Report:

              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        20
        ba00       1.00      1.00      1.00        20
   blackgram       1.00      0.95      0.97        20
    chickpea       1.00      1.00      1.00        20
     coconut       1.00      1.00      1.00        20
      coffee       1.00      1.00      1.00        20
      cotton       1.00      1.00      1.00        20
      grapes       1.00      1.00      1.00        20
        jute       0.95      1.00      0.98        20
 kidneybeans       1.00      1.00      1.00        20
      lentil       1.00      0.95      0.97        20
       maize       0.95      1.00      0.98        20
       mango       1.00      1.00      1.00        20
   mothbeans       0.95      1.00      0.98        20
    mungbean       1.00      1.00      1.00        20
   muskmelon       1.00      1.00      1.00        20
      orange    

In [21]:
fert_model = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=300,
        max_depth=20,
        random_state=42
    )
)

fert_model.fit(Xf_train, yf_train)


MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=20,
                                                     n_estimators=300,
                                                     random_state=42))

In [22]:
fert_pred = fert_model.predict(Xf_test)
mse = mean_squared_error(yf_test, fert_pred)

print("Fertilizer Model MSE:", mse)


Fertilizer Model MSE: 283.8734005178352


In [23]:
sample = pd.DataFrame([{
    "N": 90,
    "P": 40,
    "K": 42,
    "temperature": 25,
    "humidity": 80,
    "ph": 6.5,
    "rainfall": 200
}])

# Predict crop
crop_pred = crop_model.predict(sample)[0]
crop_name = le.inverse_transform([crop_pred])[0]

# Predict fertilizers
fert_result = fert_model.predict(sample)[0]

print("Recommended Crop:", crop_name)
print("\nFertilizer Recommendation:")
print("Urea:", fert_result[0])
print("SSP:", fert_result[1])
print("MOP:", fert_result[2])
print("TSP:", fert_result[3])
print("DAP:", fert_result[4])


Recommended Crop: rice

Fertilizer Recommendation:
Urea: 137.6660000000005
SSP: 156.24789333333283
MOP: 70.0
TSP: 0.0
DAP: 75.51441333333315


In [24]:
joblib.dump(crop_model, '/content/crop_model.pkl')
joblib.dump(fert_model, '/content/fert_model.pkl')
joblib.dump(le, '/content/label_encoder.pkl')
print("Models saved!")


Models saved!


In [25]:
crop_model = joblib.load('/content/crop_model.pkl')
fert_model = joblib.load('/content/fert_model.pkl')
le = joblib.load('/content/label_encoder.pkl')


In [26]:
pip freeze > requirements.txt